In [1]:
import pandas as pd; pd.set_option('max_rows', 1000)
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import yfinance as yf
import bt
import talib

# read

In [2]:
start_date = '2003-07-29'
end_date   = '2021-09-29'

indice_tickers = ['^KS11', '^GSPC', '^IXIC',  '^N225',  '^HSI',     '^FCHI', '^GDAXI']
etf_tickers    = ['EWY',   'SPY',   'QQQ',    'EWJ',    'EWH',      'EWQ',   'EWG', 'AGG']
indice_names   = ['kospi', 'sp500', 'nasdaq', 'nikkei', 'hangseng', 'cac',   'dax']
etf_names      = ['kospi', 'sp500', 'nasdaq', 'nikkei', 'hangseng', 'cac',   'dax', 'agg']

indice = yf.download(indice_tickers, start=start_date, end=end_date)
etf    = yf.download(etf_tickers, start=start_date, end=end_date)

[*********************100%***********************]  7 of 7 completed
[*********************100%***********************]  8 of 8 completed


In [3]:
indice['Adj Close']

,^FCHI,^GDAXI,^GSPC,^HSI,^IXIC,^KS11,^N225
Date,,,,,,,
2003-07-28,3164.929932,3417.770020,996.520020,10134.879883,1735.359985,NaN,NaN
2003-07-29,3142.389893,3428.120117,989.280029,10198.599609,1731.369995,722.330017,9834.309570
2003-07-30,3172.510010,3429.030029,987.489990,10121.219727,1720.910034,714.150024,9632.660156
2003-07-31,3210.270020,3487.860107,990.309998,10134.830078,1735.020020,713.520020,9563.209961
2003-08-01,3169.629883,3438.889893,980.150024,10248.599609,1715.619995,727.260010,9611.669922
...,...,...,...,...,...,...,...
2021-09-22,6637.000000,15506.740234,4395.640137,NaN,14896.849609,NaN,29639.400391
2021-09-23,6701.979980,15643.969727,4448.979980,24510.980469,15052.240234,3127.580078,NaN
2021-09-24,6638.459961,15531.750000,4455.479980,24192.160156,15047.700195,3125.239990,30248.810547


In [4]:
etf['Adj Close']

,AGG,EWG,EWH,EWJ,EWQ,EWY,QQQ,SPY
Date,,,,,,,,
2003-07-28,NaN,8.589716,4.700956,24.707077,10.765282,17.662930,27.658077,69.914536
2003-07-29,NaN,8.562790,4.724344,24.390320,10.625310,17.903793,27.449930,69.592522
2003-07-30,NaN,8.475277,4.712650,23.883501,10.695293,17.767305,27.250456,69.424454
2003-07-31,NaN,8.549325,4.765272,24.073555,10.708018,17.422075,27.580027,69.585533
2003-08-01,NaN,8.414690,4.741885,23.788481,10.517146,17.767305,27.285141,68.969398
...,...,...,...,...,...,...,...,...
2021-09-22,115.815758,33.860001,24.340000,72.489998,38.189999,81.989998,369.570007,437.859985
2021-09-23,115.247543,34.320000,24.219999,72.910004,38.860001,83.959999,372.980011,443.179993
2021-09-24,115.048172,33.970001,23.590000,72.660004,38.410000,82.510002,373.329987,443.910004


# 지수와 지수추종 ETF의 상관관계

In [5]:
pd.concat([indice['Adj Close'], etf['Adj Close']], axis=1).corr().iloc[:7, -7:]

,EWG,EWH,EWJ,EWQ,EWY,QQQ,SPY
^FCHI,0.756617,0.647044,0.888781,0.880343,0.657951,0.644941,0.701003
^GDAXI,0.951660,0.973145,0.900323,0.906903,0.887160,0.885311,0.938480
^GSPC,0.865952,0.920883,0.931032,0.896461,0.836425,0.981121,0.998247
^HSI,0.918573,0.907584,0.759410,0.858014,0.892602,0.668527,0.731338
^IXIC,0.819560,0.887665,0.898105,0.856513,0.820044,0.998858,0.991390
^KS11,0.920509,0.913245,0.777692,0.845373,0.977151,0.791617,0.817217
^N225,0.838509,0.829640,0.975932,0.903471,0.755079,0.872063,0.910902


# 글라이드 패스

In [6]:
def makeGlidePath(etf_ohlcv, glide_weight, risk_weight, start_year=2011):
    
    # 날짜, 위험자산 비중 데이터프레임
    df = pd.DataFrame(
        {
            'Date': [str(year) + '-09-29' for year in range(start_year, 2022)], 
            'RA'  : glide_weight
        }
    )
    
    # 장이 열린 날짜
    df['Date'] = df.apply(lambda x: etf_ohlcv['Adj Close'].index[etf_ohlcv['Adj Close'].index.get_loc(x['Date'], method='nearest')], axis=1)
    
    # 비위험자산 비중
    df['AGG'] = 1 - df['RA']
    
    # 위험자산 비중
    risk_etfs = ['EWG', 'EWH', 'EWJ', 'EWQ', 'EWY', 'QQQ', 'SPY']
    for risk_etf, weight in zip(risk_etfs, risk_weight):
        df[risk_etf] = df['RA'] * weight
    
    df.set_index('Date', inplace=True)
    df.drop('RA', axis=1, inplace=True)
    
    return df

In [7]:
# 위험자산 한글명 [독일,   홍콩,  일본,   프랑스, 한국,   S&P,   나스닥]
# 위험자산내 비중 ['EWG', 'EWH', 'EWJ', 'EWQ', 'EWY', 'QQQ', 'SPY']
risk_weight   = [.0250, .0250, .0375, .0375, .2500, .2000, .4250]

In [8]:
# 변동성 학습-예측 데이터의 글라이드 패스
glide_weight_baseline = [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
gp_train = makeGlidePath(etf, glide_weight_baseline, risk_weight, start_year=2003)
gp_train

,AGG,EWG,EWH,EWJ,EWQ,EWY,QQQ,SPY
Date,,,,,,,,
2003-09-29,0,0.025,0.025,0.0375,0.0375,0.25,0.2,0.425
2004-09-29,0,0.025,0.025,0.0375,0.0375,0.25,0.2,0.425
2005-09-29,0,0.025,0.025,0.0375,0.0375,0.25,0.2,0.425
2006-09-29,0,0.025,0.025,0.0375,0.0375,0.25,0.2,0.425
2007-09-28,0,0.025,0.025,0.0375,0.0375,0.25,0.2,0.425
2008-09-29,0,0.025,0.025,0.0375,0.0375,0.25,0.2,0.425
2009-09-29,0,0.025,0.025,0.0375,0.0375,0.25,0.2,0.425
2010-09-29,0,0.025,0.025,0.0375,0.0375,0.25,0.2,0.425
2011-09-29,0,0.025,0.025,0.0375,0.0375,0.25,0.2,0.425


# 백테스트를 통해 학습할 변동성 데이터 수집

In [9]:
train_data = etf['Adj Close'].loc['2003-07-29':]

train_strategy = bt.Strategy(
    'train_portfolio', 
    algos=[
        bt.algos.
        bt.algos.SelectAll(), 
        bt.algos.WeighTarget(gp_train), 
        bt.algos.Rebalance(), 
    ]
)
train_backtest = bt.Backtest(strategy=train_strategy, data=train_data)
train_result   = bt.run(train_backtest)

In [10]:
train_hv = train_result.prices
train_hv['HV'] = train_hv['train_portfolio'].pct_change().rolling(window=20, min_periods=0, center=False).std() * np.sqrt(252)
train_hv = train_hv['2003-09-29':]
train_hv

,train_portfolio,HV
2003-09-29,100.000000,0.000000
2003-09-30,98.666182,0.047346
2003-10-01,101.496201,0.114521
2003-10-02,102.256298,0.116916
2003-10-03,104.068600,0.130730
...,...,...
2021-09-22,664.495918,0.116953
2021-09-23,673.861871,0.128578
2021-09-24,670.686647,0.126863
2021-09-27,669.888577,0.120544


In [11]:
train_result.get_security_weights()['2003-09-29':]

,EWG,EWH,EWJ,EWQ,EWY,QQQ,SPY
2003-09-29,0.024993,0.024997,0.037495,0.037492,0.249992,0.199988,0.424971
2003-09-30,0.024951,0.025282,0.037488,0.037756,0.249630,0.198288,0.426534
2003-10-01,0.025170,0.025303,0.037940,0.037585,0.253354,0.197100,0.423478
2003-10-02,0.024500,0.025811,0.038402,0.036986,0.256096,0.196284,0.421853
2003-10-03,0.024662,0.025766,0.038423,0.037223,0.257290,0.198258,0.418310
...,...,...,...,...,...,...,...
2021-09-22,0.022599,0.021769,0.035160,0.039466,0.243701,0.205003,0.432262
2021-09-23,0.022588,0.021360,0.034872,0.039600,0.246088,0.204019,0.431433
2021-09-24,0.022463,0.020903,0.034917,0.039327,0.242983,0.205177,0.434189
2021-09-27,0.022523,0.021079,0.034930,0.039466,0.244717,0.203782,0.433463


# 보조지표

## 사전정의함수

In [12]:
def getPsy(x):
    """심리도 사전 정의 함수"""
    if x > 0:
        result = 1
    else:
        result = np.nan
    return result

def getRevPlus(a):
    if a > 0:
        result = a
    else:
        result = 0
    return result

def getRevMinus(a):
    if a < 0:
        result = a
    else:
        result = 0
    return result

def getVRplus(a, b):
    if a > 0:
        result = b
    else:
        result = 0
    return result

def getVRminus(a, b):
    if a < 0:
        result = b
    else:
        result = 0
    return result

## 보조지표 함수

In [13]:
def getIndicators(df):
    
    # Change : 변동률
    df['Change'] = df['Adj Close'].pct_change()
    
    # Moving Average : 주가이동평균
    df['MA5'] = talib.MA(df['Adj Close'], timeperiod=5)
    df['MA20'] = talib.MA(df['Adj Close'], timeperiod=20)
    df['MA60'] = talib.MA(df['Adj Close'], timeperiod=60)
    df['MA120'] = talib.MA(df['Adj Close'], timeperiod=120)
    # 가격이동평균 기울기 : (당일 이동평균 - 전날 이동평균) / 전날 이동평균
    df['MA5_gra'] = (df['MA5'] - df['MA5'].shift(1)) / df['MA5'].shift(1)
    df['MA20_gra'] = (df['MA20'] - df['MA20'].shift(1)) / df['MA20'].shift(1)
    df['MA60_gra'] = (df['MA60'] - df['MA60'].shift(1)) / df['MA60'].shift(1)
    df['MA120_gra'] = (df['MA120'] - df['MA120'].shift(1)) / df['MA120'].shift(1)    
    
    # 거래량이동평균
    df['V_MA5'] = talib.MA(df['Volume'], timeperiod=5)
    df['V_MA20'] = talib.MA(df['Volume'], timeperiod=20)
    df['V_MA60'] = talib.MA(df['Volume'], timeperiod=60)
    df['V_MA120'] = talib.MA(df['Volume'], timeperiod=120)
    
    # 지수이동평균
    df['EMA5'] = talib.EMA(df['Adj Close'], timeperiod=5)
    df['EMA20'] = talib.EMA(df['Adj Close'], timeperiod=20)
    df['EMA60'] = talib.EMA(df['Adj Close'], timeperiod=60)
    df['EMA120'] = talib.EMA(df['Adj Close'], timeperiod=120)   
    
    # MACD
    macd, macdsignal, macdhist = talib.MACD(df['Adj Close'], fastperiod=12, slowperiod=26, signalperiod=9)
    df['MACD'] = macd
    df['MACD_sig9'] = macdsignal
    df['MACD_OSC'] = macdhist
    
    # Stochastic
    fastk, fastd = talib.STOCHF(df['High'], df['Low'], df['Adj Close'], fastk_period=14, fastd_period=3)
    slowk, slowd = talib.STOCH(df['High'], df['Low'], df['Adj Close'], fastk_period=14, slowk_period=5, slowd_period=3)
    df['Fast%K'] = fastk
    df['Fast%D'] = fastd
    df['Slow%K'] = slowk
    df['Slow%D'] = slowd
    
    # RSI: 9일, 14일, 15일, 25일, 28일 (RSI_sig 9일)
    df_index = df.index.astype('str')
    U = np.where(df['Adj Close'].diff(1) > 0, df['Adj Close'].diff(1), 0)
    D = np.where(df['Adj Close'].diff(1) < 0, df['Adj Close'].diff(1)*(-1), 0)
    AU = pd.DataFrame(U, index = df_index).rolling(window=14).mean() # RSI: 14일
    AD = pd.DataFrame(D, index = df_index).rolling(window=14).mean()
    RSI = AU / (AD+AU) * 100
    
    df.insert(len(df.columns), "RSI14", RSI)
    df.insert(len(df.columns), "RSI signal", df['RSI14'].ewm(span=9).mean())
    
    # CCI (14일 기준)
    M = (df['High']+df['Low']+df['Adj Close'])/3
    m = M.rolling(window=14).mean()
    d = abs(M-m).rolling(window=14).mean()
    df['CCI'] =( M - m )/(d*0.015)
    df['CCI'] = np.where(df['CCI'] > 0, df['CCI'] + 0.5, df['CCI'] - 0.5)
    
    # A ratio
    df['A'] = (df['High'] - df['Open']).rolling(window=20).sum() / (df['Open'] - df['Low']).rolling(window=20).sum()
    df['A_Ratio'] = df['A']*100
    
    # B ratio
    df['B'] = (df['High'] - df['Adj Close'].shift(1)).rolling(window=20).sum() / (df['Adj Close'].shift(1) - df['Low']).rolling(window=20).sum()
    df['B_Ratio'] = df['B']*100
    
    
    df.drop(['A', 'B'], axis=1, inplace=True)
    
    # ATR(14일 기준)
    df['ATR'] = talib.ATR(df['High'], df['Low'], df['Adj Close'], timeperiod=14)
    
    # DMI
    df['UpDI'] = talib.PLUS_DI(df['High'], df['Low'],  df['Adj Close'], timeperiod=14)
    df['DownDI'] = talib.MINUS_DI(df['High'], df['Low'],  df['Adj Close'], timeperiod=14)
    
    # MFI
    df['MFI'] = talib.MFI(df['High'], df['Low'], df['Adj Close'], df['Volume'], timeperiod=14)
    
    # Momentum
    df['Momentum'] = talib.MOM(df['Adj Close'], timeperiod=12)
    
    # Sonar
    df['EMA9'] = talib.EMA(df['Adj Close'], timeperiod=9)
    df['n_shift_ema'] = df['EMA9'].shift(9)
    df['Sonar'] = df['EMA9'] - df['n_shift_ema']
    df['Sonar_sig5'] = talib.EMA(df['Sonar'], timeperiod=5)
    
    df.drop('EMA9', axis=1, inplace= True)
    
    df.drop('n_shift_ema', axis=1, inplace=True)
    
    # STDEV(20일 표준편차)
    df['STDEV'] = talib.STDDEV(df['Adj Close'], timeperiod=20)
    
    # Sigma(20일) (20일짜리 표준편차 사용)
    df['Sigma'] = ( df['Adj Close'] - df['Adj Close'].rolling(window=20).mean() ) / df['STDEV']
    
    # TRIX(12일) (퍼센트)
    df['TRIX'] = talib.TRIX(df['Adj Close'], timeperiod=12)
    
    # VR : 주가 상승일 거래량 / 주가 하락일 거래량 * 100, 
    df['VRplus']  = df.apply(lambda x: getVRplus(x['Change'], x['Volume']), axis=1)
    df['VRminus'] = df.apply(lambda x: getVRminus(x['Change'], x['Volume']), axis=1)
    
    df['VRplus_rolling']  = df['VRplus'].rolling(window=20).sum() # period: 20
    df['VRminus_rolling'] = df['VRminus'].rolling(window=20).sum()
    
    df['VR'] = df['VRplus_rolling'] / df['VRminus_rolling'] * 100
    
    df.drop(['VRplus', 'VRminus', 'VRplus_rolling', 'VRminus_rolling'], axis=1, inplace=True)
    
    # William's %R (14일)
    df['WILLR'] = talib.WILLR(df['High'], df['Low'], df['Adj Close'], timeperiod=14)
    
    # 심리도
    df['psy_temp'] = df.apply(lambda x: getPsy(x['Change']), axis=1)
    df['Psy']      = df['psy_temp'].rolling(window=10).count() / 10 * 100
    df.drop('psy_temp', axis=1, inplace=True)
    
    # 이격도
    df['Disparity20'] = (df['Adj Close'] / df['Adj Close'].rolling(window=20).mean()) * 100
    df['Disparity60'] = (df['Adj Close'] / df['Adj Close'].rolling(window=60).mean()) * 100
    
    # ADX
    df['ADX'] = talib.ADX(df['High'], df['Low'], df['Adj Close'], timeperiod=14)
    
    # Chaikin's OSC
    df['ADOSC'] = talib.ADOSC(df['High'], df['Low'], df['Adj Close'], df['Volume'], fastperiod=3, slowperiod=10)
    
    # 거래량 OSC(10일, 20일, 9일)
    df['Volume_OSC'] = (df['Volume'].rolling(window=10).mean() - df['Volume'].rolling(window=20).mean()) / df['Volume'].rolling(window=10).mean() * 100
    
    # 볼린저 밴드
    df['upperband'], df['middleband'], df['lowerband'] = talib.BBANDS(df['Adj Close'], timeperiod=20, nbdevup=2, nbdevdn=2, matype=0)
    
    
    return df

In [14]:
etfs = []
# ['EWG', 'EWH', 'EWJ', 'EWQ', 'EWY', 'QQQ', 'SPY']
for i, j in zip(etf['Adj Close'].columns[1:], risk_weight):
    temp_etf = getIndicators(etf.swaplevel(axis=1)[i])['2003-09-29':]
    temp_etf.fillna(0, inplace=True)
    temp_etf = temp_etf.loc[:, 'MA5':] * j
    etfs.append(temp_etf)

f:\workspace\venv\lib\site-packages\pandas\core\window\rolling.py:1606: FutureWarning: min_periods=None will default to the size of window consistent with other methods in a future version. Specify min_periods=0 instead.
  warnings.warn(
f:\workspace\venv\lib\site-packages\pandas\core\window\rolling.py:1606: FutureWarning: min_periods=None will default to the size of window consistent with other methods in a future version. Specify min_periods=0 instead.
  warnings.warn(
f:\workspace\venv\lib\site-packages\pandas\core\window\rolling.py:1606: FutureWarning: min_periods=None will default to the size of window consistent with other methods in a future version. Specify min_periods=0 instead.
  warnings.warn(
f:\workspace\venv\lib\site-packages\pandas\core\window\rolling.py:1606: FutureWarning: min_periods=None will default to the size of window consistent with other methods in a future version. Specify min_periods=0 instead.
  warnings.warn(
f:\workspace\venv\lib\site-packages\pandas\core\

In [15]:
sum(etfs)

,MA5,MA20,MA60,MA120,MA5_gra,MA20_gra,MA60_gra,MA120_gra,V_MA5,V_MA20,...,WILLR,Psy,Disparity20,Disparity60,ADX,ADOSC,Volume_OSC,upperband,middleband,lowerband
Date,,,,,,,,,,,,,,,,,,,,,
2003-09-29,42.146220,42.842343,0.000000,0.000000,-0.003142,-0.000810,0.000000,0.000000,3.822771e+07,3.386586e+07,...,-472.336735,38.250,97.591746,0.000000,18.762568,-3.909975e+09,0.484744,44.066591,42.842343,41.618094
2003-09-30,41.847139,42.769253,0.000000,0.000000,-0.007730,-0.002238,0.000000,0.000000,4.313977e+07,3.470818e+07,...,-450.208606,28.250,96.506009,0.000000,19.247360,-4.059619e+09,4.466314,44.108399,42.769253,41.430107
2003-10-01,41.927371,42.740050,0.000000,0.000000,0.002179,-0.000942,0.000000,0.000000,4.484279e+07,3.536592e+07,...,-424.533872,37.875,99.353204,0.000000,18.627031,-4.085364e+09,8.823151,44.074615,42.740050,41.405484
2003-10-02,42.126826,42.717888,0.000000,0.000000,0.005389,-0.000766,0.000000,0.000000,4.280619e+07,3.591528e+07,...,-415.254148,40.125,100.166140,0.000000,17.712021,-4.208542e+09,8.880720,44.038391,42.717888,41.397384
2003-10-03,42.477221,42.737590,0.000000,0.000000,0.009944,0.000424,0.000000,0.000000,4.357655e+07,3.650078e+07,...,-395.685218,49.500,101.891170,0.000000,17.789960,-4.319474e+09,9.854309,44.081504,42.737590,41.393676
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-09-22,286.640851,291.411425,287.871643,279.839167,-0.004953,-0.001046,-0.000220,0.000377,5.936192e+07,3.988466e+07,...,-65.606015,37.375,97.993628,98.261874,21.025450,-6.262364e+07,17.145034,298.973921,291.411425,283.848930
2021-09-23,286.235189,291.293727,288.016230,280.035162,-0.000978,-0.000422,0.000009,0.000371,5.966120e+07,4.090081e+07,...,-44.815223,46.750,99.414129,99.629497,20.712005,-4.633938e+07,16.877608,298.868582,291.293727,283.718871
2021-09-24,286.409275,291.269103,288.160017,280.232441,-0.000056,-0.000286,-0.000058,0.000344,5.384071e+07,4.106236e+07,...,-48.692807,52.750,98.956341,99.163220,20.596428,-2.923581e+07,15.228066,298.848010,291.269103,283.690197


In [16]:
train_multivar = train_hv.merge(sum(etfs), how='left', left_index=True, right_index=True)

In [17]:
train_multivar

,train_portfolio,HV,MA5,MA20,MA60,MA120,MA5_gra,MA20_gra,MA60_gra,MA120_gra,...,WILLR,Psy,Disparity20,Disparity60,ADX,ADOSC,Volume_OSC,upperband,middleband,lowerband
2003-09-29,100.000000,0.000000,42.146220,42.842343,0.000000,0.000000,-0.003142,-0.000810,0.000000,0.000000,...,-472.336735,38.250,97.591746,0.000000,18.762568,-3.909975e+09,0.484744,44.066591,42.842343,41.618094
2003-09-30,98.666182,0.047346,41.847139,42.769253,0.000000,0.000000,-0.007730,-0.002238,0.000000,0.000000,...,-450.208606,28.250,96.506009,0.000000,19.247360,-4.059619e+09,4.466314,44.108399,42.769253,41.430107
2003-10-01,101.496201,0.114521,41.927371,42.740050,0.000000,0.000000,0.002179,-0.000942,0.000000,0.000000,...,-424.533872,37.875,99.353204,0.000000,18.627031,-4.085364e+09,8.823151,44.074615,42.740050,41.405484
2003-10-02,102.256298,0.116916,42.126826,42.717888,0.000000,0.000000,0.005389,-0.000766,0.000000,0.000000,...,-415.254148,40.125,100.166140,0.000000,17.712021,-4.208542e+09,8.880720,44.038391,42.717888,41.397384
2003-10-03,104.068600,0.130730,42.477221,42.737590,0.000000,0.000000,0.009944,0.000424,0.000000,0.000000,...,-395.685218,49.500,101.891170,0.000000,17.789960,-4.319474e+09,9.854309,44.081504,42.737590,41.393676
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-09-22,664.495918,0.116953,286.640851,291.411425,287.871643,279.839167,-0.004953,-0.001046,-0.000220,0.000377,...,-65.606015,37.375,97.993628,98.261874,21.025450,-6.262364e+07,17.145034,298.973921,291.411425,283.848930
2021-09-23,673.861871,0.128578,286.235189,291.293727,288.016230,280.035162,-0.000978,-0.000422,0.000009,0.000371,...,-44.815223,46.750,99.414129,99.629497,20.712005,-4.633938e+07,16.877608,298.868582,291.293727,283.718871
2021-09-24,670.686647,0.126863,286.409275,291.269103,288.160017,280.232441,-0.000056,-0.000286,-0.000058,0.000344,...,-48.692807,52.750,98.956341,99.163220,20.596428,-2.923581e+07,15.228066,298.848010,291.269103,283.690197
2021-09-27,669.888577,0.120544,287.440325,291.056016,288.272083,280.419473,0.003800,-0.000856,-0.000096,0.000338,...,-47.885598,48.375,98.931764,99.051161,20.533357,-2.072008e+07,13.064686,298.666889,291.056016,283.445142


In [18]:
# train_multivar.to_csv('./train_multivar.csv')